### Identifier et traiter les valeurs manquantes


Dans cette étape, On:
- Importe les données
- Supprime la colonne des IDs
- Remplace les valeur manquantes par NaN, afin d'etre identifier par KNNImputer
- Garde les valeurs nuls (les zeros) de la colonne Pregnancies, parce que c'est logique
- Utilise KNNImputer pour remplacer les valeurs manquantes
- On affiche le nombre de valeurs nuls, pour confirmer le success de remplacement


In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
import os
import numpy as np

df = pd.read_csv(os.getcwd() + '/../data/raw/dataset-diabete.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)

df["Glucose"] = df["Glucose"].replace(0, np.nan)
df["BloodPressure"] = df["BloodPressure"].replace(0, np.nan)
df["SkinThickness"] = df["SkinThickness"].replace(0, np.nan)
df["Insulin"] = df["Insulin"].replace(0, np.nan)
df["BMI"] = df["BMI"].replace(0, np.nan)

imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

print(df.isnull().sum(), '\n\n')